In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import re
import time

In [2]:
# Read in DataFrame
song_lyrics_df = pd.read_csv('../Data/song_lyrics.csv')
song_lyrics_df = song_lyrics_df.drop('non_alpha_words', axis=1)
song_lyrics_df.head(1)

,song,song_id,artist,artist_id,category,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...


In [6]:
# Create DataFrame of columns from Spotify API
spotify_df = song_lyrics_df.drop('lyrics', axis=1)
spotify_df

,song,song_id,artist,artist_id,category,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.83300,0.001790,0.1450,0.529,81.112,214707.0,4.0
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.08050,0.000000,0.3660,0.756,179.954,206046.0,4.0
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.21300,0.000000,0.1340,0.467,93.005,161385.0,4.0
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,-6.900,1.0,0.0944,0.44000,0.000007,0.0921,0.531,80.870,238805.0,4.0
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,-5.905,1.0,0.0373,0.31800,0.000000,0.4140,0.329,140.013,185850.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8977,Louisiana Saturday Night,50PU05RTGva8laKDwxED9Y,Mel McDaniel,1fMdO9vLUd6wW4jbMeNiup,country,"['classic', 'oklahoma', 'country']",0.686,0.622,2.0,-7.310,1.0,0.0857,0.53200,0.000000,0.0511,0.982,163.245,144187.0,4.0
8978,Free And Easy (Down The Road I Go),4Ng7hGdPwpINleVecBWiOO,Dierks Bentley,7x8nK0m0cP2ksQf0mjWdPS,country,"['contemporary', 'country', 'modern']",0.557,0.825,6.0,-4.657,1.0,0.0283,0.00525,0.001640,0.1770,0.841,120.017,201120.0,4.0
8979,I Love This Bar,32jHB7W9b08OTorAqR0cJo,Toby Keith,2bA6fzP0lMAQ4kz6CF61w8,country,"['oklahoma', 'contemporary', 'country']",0.608,0.825,2.0,-6.303,1.0,0.0419,0.02400,0.006560,0.3460,0.454,115.049,335333.0,4.0
8980,Roll On (Eighteen Wheeler),31LOzuiEqzfMZ6f8nujDVd,Alabama,6rJqqRce0Kvo2dJUXoHleC,country,['country'],0.584,0.456,5.0,-19.116,1.0,0.0578,0.11500,0.000000,0.5560,0.640,131.814,262627.0,4.0


In [4]:
# Create DataFrame of columns from Genius API
genius_df = song_lyrics_df.drop(['song_id', 'artist_id', 'category', 'genres'], axis=1)
genius_df = genius_df.drop(genius_df.columns[2:15], axis=1)
genius_df

,song,artist,lyrics
0,willow,Taylor Swift,Im like the water when your ship rolled in th...
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,Didnt even wanna go out whyd you call me ? Iv...
2,WITHOUT YOU,The Kid LAROI,You cut out a piece of me and now I bleed int...
3,Heat Waves,Glass Animals,Last night all I think about is you Dont stop...
4,r u ok,Tate McRae,Are you okay ? Cause youre the one who needed...
...,...,...,...
8977,Louisiana Saturday Night,Mel McDaniel,Well you get down the fiddle and you get down ...
8978,Free And Easy (Down The Road I Go),Dierks Bentley,Pair of boots and a sack of clothes Free and ...
8979,I Love This Bar,Toby Keith,We got winners we got losers Chain smokers an...
8980,Roll On (Eighteen Wheeler),Alabama,Roll on highway roll on along Roll on daddy t...


In [5]:
# Save DataFrames to CSV files
spotify_df.to_csv('../Data/spotify_df.csv', index=False)
genius_df.to_csv('../Data/genius_df.csv', index=False)